# DSCI 6003 5.2 Lecture 


## By the End of this Lecture You Will Be:
1. Be more confident with decision trees
2. Be able to describe in your own words what a random forest is
3. Have a basic sense of how to build a random forest.


### Reviewing the Tree Construction Pseudocode

    Construct root node
        while X is larger than one row (or has not hit stop condition)
            find set S = {S1, S2} that minimizes impurities
            choose the S that minimizes the size
            construct child nodes 1 and 2 and pass them S1, S2

## Characterizing the Output of Trees

### Classification Trees

The decision boundaries of classification trees is one of the easiest things to characterize and understand in machine learning. We can actually draw out the result of the decision process in terms of a set of subdivisions of the output space:

![tree-appearance](./images/DecisionTree_2.gif)

Each split of the tree can be characterized as a simple line dividing all points with the selected attribute along that axis to points above and below the selection. This image is described as the **partitioning** of the input space, where the boundaries of each domain are determined based on the branch points of the tree. The partitioning of a correctly-built classification tree amounts to a continuous stepped line showing the boundary between the two classes, with each corner on the step describing the boundary of that tree split.

### Regression Trees

Like many - if not most - standard **classifiers**, trees and by extension, forests, can be converted into **regressors** and thus used for continuous predictions. In the case of single trees, the tree parses the inputs into large domains, and produces a single output (based on the training data) as an output of that domain (where the domain is defined by the splits).

In the below figures ([obtained from lecture notes here](http://www.stat.cmu.edu/~cshalizi/350-2006/lecture-10.pdf)) we see the construction of trees for predicting car price. Below is a standard tree regressed against horsepower and wheelbase: 

![tree1](./images/Wheelbase_tree_3.png)

Below is the partitioning of the regression:

![partition](./images/Tree_Parsing_1.png)


It's worth noting that if multiple predictor variables (features) are equally good at determining the outcome, the split chosen at that branch level is determined by chance. This is the essential concept behind the notion of *feature importance*. Here is the same tree as above, regressed against weight instead of wheelbase at the second level:

![tree1](./images/Weight_tree_2.png)








## Ensembles

*Ensemble Methods* combine multiple machine learning algorithms to obtain better predictive performance. The idea is simple: run multiple models on the data and use their predictions to make a prediction that is better than any of the models could do alone.

In the case of decision trees, we extend this cognate to Random Forests


## Bagging

Bagging, also known as *bootstrap aggregating*, is for running multiple models in parallel (the models don't use each other's results in order to predict). Each model gets a vote on the final prediction. The majority vote (again determined by a user-set parameter) is what is reported from the bagged model. Bagging is intended to address the problem of sample bias by enabling the model to capture bias within the training sample itself.

![Bagging](./images/BaggingCropped_2.png)

For classification problems (predicting a categorical value), we choose the label with the most votes.

For regression problems (predicting a continuous value), we average the values given by all the models.

You can bag with any single algorithm or collection of algorithms, giving them each a vote to the final prediction. This means you can leverage the voting power of several different kinds of models, assuming that they have similar output space (what if they do not?). Scikits have a simple Bagging API implemented so that you can bag any [regressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) or [classifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html). [This example](http://scikit-learn.org/stable/auto_examples/ensemble/plot_bias_variance.html) is a useful one to study.


## Random Forests

Probably the most common ensemble method is a *Random Forest*, which consists of a collection of Decision Trees.

They were developed by Leo Breimen, who has the most extensive notes about them on his [webpage](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm).

The idea is to repeatedly randomly select data from the dataset (*with replacement*) and build a Decision Tree with each new sample. The default is to have the randomly selected data be the same size as the initial dataset. Note that since we are sampling with replacement, many data points will be repeated in the sample and many won't be included.

Random Forests also limit each node of the Decision Tree to only consider splitting on a random subset of the features.

Here is the pseudocode for creating a Random Forest:

    CreateRandomForest(data, num_trees, num_features):
        Repeat num_trees times:
            Create a random sample of the test data with replacement
            Build a decision tree with that sample (only consider num_features features at each node)
        Return the list of the decision trees created

To classify a new document, use each tree to get a prediction. Choose the label that gets the most votes.

The default parameters that sklearn uses, which are also standard defaults, are 10 trees and only considering sqrt(m) features (where m is the total number of features).


### Out of Bag Error

We can analyze a Random Forest using the standard cross validation method of splitting the dataset into a training set and a testing set. However, if we're clever, we notice that each tree doesn't see all of the training data, so we can use the skipped data to cross validate each tree individually.

We'll skip the mathematical proof, but when selecting from the dataset, about one third of the data is left out (discussed [here](http://math.stackexchange.com/questions/203491/expected-coverage-after-sampling-with-replacement-k-times) if you want to think about the math). So every data point can be tested with about 1/3 of the trees. We calculate the percent of these that we get correct, and this is the *out-of-bag error*.

It has been proven that this is sufficient and that cross validation is not strictly necessary for a random forest, but we often still use it as that makes it easier to compare with other models.


### Feature Importance

We can use the random forest to determine which features are the most importance in predicting the class.

Breiman, the originator of random forests, uses out-of-bag error to determine feature importance, discussed [here](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#varimp). The idea is to compare the out-of-bag error of the trees with the out-of-bag error of the trees if you change the feature's value (basically, if we screw with the value of the feature, how much does that impact the total error?). Here is the pseudocode for calculating the feature importance for a single feature:

        For every tree:
            Take the data that is not covered by the tree.
            Randomly permute the values of the feature (i.e. keep the same values,
                but shuffle them around the data points).
            Calculate the OOB error on the data with the feature values permuted.
            Subtract the permutated OOB from the OOB of the original data to get the
                feature importance on this tree.
        Average all the individual feature importances to get the feature importance.

sklearn uses a different method, described [here](http://scikit-learn.org/stable/modules/ensemble.html#feature-importance-evaluation). Their method doesn't involve using the out-of-bag score. Basically, the higher in the tree the feature is, the more important it is in determining the result of a data point. The expected fraction of data points that reach a node is used as an estimate of that feature's importance for that tree. Then average those values across all trees to get the feature's importance.


### Regression Forests

Random Forests can also be used for regression. They work very similarly to Classification Random Forests, except when predicting a new value, they take the average of the predicted values.


### Random Forests with sklearn

sklearn has both Random Forests for both [classification](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [regression](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). The documentation explains all of the parameters that you have control over, but you can use it like all other sklearn models:

```python
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifer()
rf.fit(X_train, y_train)
print "accuracy:", rf.score(X_test, y_test)
```

## Resources:

* [Random Forests by Leo Breiman and Adele Cutler](http://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm)
* [sklearn ensembles](http://scikit-learn.org/stable/modules/ensemble.html)
* [Applied Data Science](http://columbia-applied-data-science.github.io/appdatasci.pdf) (Section 9.4.2)